# Case Nestlé - Data Engineer

In [1]:
# Caso não tenha o  pyspark instalado executar o seguinte comando:
# pip install pyspark

In [2]:
# Bibliotecas utilizadas
from pyspark.sql import SparkSession
from pyspark.sql.functions import count, when, col, sha2, udf, countDistinct, desc
from pyspark.sql.types import StringType, FloatType

In [3]:
# Criando uma nova sessão Spark
spark = SparkSession.builder.appName("DE").getOrCreate()

23/03/20 15:22:34 WARN Utils: Your hostname, AGM-593886.local resolves to a loopback address: 127.0.0.1; using 192.168.100.6 instead (on interface en0)
23/03/20 15:22:34 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/20 15:22:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Análise exploratória das bases

In [4]:
# Input DataFrames
cargos_df = spark.read.format("csv").option("header", "true").option("delimiter", ";").load("./Base_case/BaseCargos.csv")
cep_df = spark.read.format("csv").option("header", "true").option("delimiter", "|").load("./Base_case/BaseCEP.csv")
clientes_df = spark.read.format("csv") \
                      .option("header", "true") \
                      .option("delimiter", ";") \
                      .load("./Base_case/BaseClientes.csv")
funcionarios_df = spark.read.format("csv").option("header", "true").option("delimiter", "||").load("./Base_case/BaseFuncionarios.csv")
nivel_df = spark.read.format("csv").option("header", "true").option("delimiter", "%").load("./Base_case/BaseNivel.csv")
pq_df = spark.read.format("csv").option("header", "true").option("delimiter", ",").load("./Base_case/BasePQ.csv")

In [5]:
#Função para retornar os 5 primeiros valores de cada df
def show_df(df):
    return df.show(5)

In [6]:
show_df(cargos_df)

+-----+----------+--------------+--------+---------+--------+-----+-----------+
|Cargo|     Nível|          Área|COD Área|COD Nível|  Quadro|Bonus|Contratacao|
+-----+----------+--------------+--------+---------+--------+-----+-----------+
|  OPV|   Diretor|     Operações|     JAJ|       JE| Efetivo|    S|  Diretoria|
|  LOI|Estagiário|     Logísitca|     EDE|       JA| Efetivo|    N|    Gerente|
|  ADI|Estagiário|Administrativo|     BAC|       JA| Efetivo|    N|    Gerente|
| ADII|  Analista|Administrativo|     BAC|       DB|Terceiro|    N|         RH|
| OPII|  Analista|     Operações|     JAJ|       DB|Terceiro|    N|         RH|
+-----+----------+--------------+--------+---------+--------+-----+-----------+
only showing top 5 rows



In [7]:
# show_df(cep_df)

In [8]:
show_df(clientes_df)

23/03/20 15:22:49 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Cliente, Valor Contrato Anual, Quantidade de Serviços, Cargo Responsável, CEP, Data Início Contrato, Nivel de Importancia, 
 Schema: Cliente, Valor Contrato Anual, Quantidade de Serviços, Cargo Responsável, CEP, Data Início Contrato, Nivel de Importancia, _c7
Expected: _c7 but found: 
CSV file: file:///Users/raquel.bustamante/Documents/Nestlé/Base_case/BaseClientes.csv
+------------------+--------------------+----------------------+-----------------+---------+--------------------+--------------------+----+
|           Cliente|Valor Contrato Anual|Quantidade de Serviços|Cargo Responsável|      CEP|Data Início Contrato|Nivel de Importancia| _c7|
+------------------+--------------------+----------------------+-----------------+---------+--------------------+--------------------+----+
|Teixeira Gonçalves|               54000|                    37|            ADIII|743419711|          15/03/2019| 

In [9]:
# show_df(funcionarios_df)

In [10]:
show_df(nivel_df)

+-----+---------------+--------------+---------------+------------------+-----------------+-----------------+
|Nível|Descrição Nível|Tempo no Nível| Plano de Saúde|Plano Odontológico|Setor Responsável|Plano de Carreira|
+-----+---------------+--------------+---------------+------------------+-----------------+-----------------+
|   JA|     Estagiário|             1|           null|              null|             null|             null|
| null|            Sim|           Não|RH Universidade|               Sim|             null|             null|
|   DB|       Analista|             4|           null|              null|             null|             null|
|  Sim|            Não|   RH Empresas|            Sim|              null|             null|             null|
|   GC|    Coordenador|             5|           null|              null|             null|             null|
+-----+---------------+--------------+---------------+------------------+-----------------+-----------------+
only showi

In [11]:
# show_df(pq_df)

In [12]:
# Função para análise de quantidade de registros e colunas em cada base
dfs = [
    ("Cargos", cargos_df), 
    ("Base CEP", cep_df), 
    ("Base Clientes", clientes_df,),
    ("Base Funcionarios", funcionarios_df), 
    ("Base Nivel", nivel_df), 
    ("Base PQ", pq_df)
]

for base, df in dfs:
    print(f'{base} tem {df.count()} registros e {len(df.columns)} colunas')

    print(f"Valores nulos em {base}:")
    df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).show()

Cargos tem 25 registros e 8 colunas
Valores nulos em Cargos:
+-----+-----+----+--------+---------+------+-----+-----------+
|Cargo|Nível|Área|COD Área|COD Nível|Quadro|Bonus|Contratacao|
+-----+-----+----+--------+---------+------+-----+-----------+
|    0|    0|   0|       0|        0|     0|    0|          0|
+-----+-----+----+--------+---------+------+-----+-----------+

Base CEP tem 860 registros e 3 colunas
Valores nulos em Base CEP:
+---+------+------+
|CEP|Estado|Região|
+---+------+------+
|  0|    25|     0|
+---+------+------+

Base Clientes tem 321 registros e 8 colunas
Valores nulos em Base Clientes:
23/03/20 15:22:53 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Cliente, Valor Contrato Anual, Quantidade de Serviços, Cargo Responsável, CEP, Data Início Contrato, Nivel de Importancia, 
 Schema: Cliente, Valor Contrato Anual, Quantidade de Serviços, Cargo Responsável, CEP, Data Início Contrato, Nivel de Importancia, _c7
Expected: _c7 but found: 
C

## Tratativa dados sensíveis

Foi escolhida a coluna CPF das bases funcionários e PQ para serem tratadas como dados sensíveis. Para isso, é possível utilizar a função sha2 para substituir os valores por hash.

In [13]:
funcionarios_df_hash = funcionarios_df.withColumn("CPF", sha2(col('CPF'), 256))

In [14]:
pq_df_hash = pq_df.withColumn("CPF", sha2(col('CPF'), 256))

## Insights

Baseado na análise exploratória feita sobre as tabelas quanto ao seus devidos conteúdos, foi escolhido gerar 5 insights sobre os dados
1. Quantidade de clientes e funcionários por estado
2. Distribuição de cargos por nível
3. Funcionários com maiores horas extras da base PQ
4. Quantidade de Clientes e de Valor Contrato Anual por Nível (Dado o cargo responsável)
5. Clientes com maior Valor Contrato Anual

#### Quantidade de clientes e funcionários por estado

In [15]:
# Criando os dfs
clientes_estado_df = clientes_df.join(cep_df, "CEP").groupBy("Estado").count().withColumnRenamed("count", "total_clientes").orderBy("total_clientes", ascending=False)
funcionarios_estado_df = funcionarios_df.join(cep_df, "CEP").groupBy("Estado").count().withColumnRenamed("count", "total_funcionarios").orderBy("total_funcionarios", ascending=False)

In [16]:
# Unindo os dfs e ordenando por estados com mais clientes e funcionários
clientes_funcionarios_estado = clientes_estado_df.join(funcionarios_estado_df, "Estado", "outer")\
    .orderBy(["total_funcionarios","total_clientes"], ascending=False)

In [17]:
clientes_funcionarios_estado.show()

+-------------------+--------------+------------------+
|             Estado|total_clientes|total_funcionarios|
+-------------------+--------------+------------------+
|            Alagoas|            22|                41|
|           Amazonas|            24|                37|
|               Pará|            17|                35|
|        Mato Grosso|            15|                35|
|            Sergipe|            20|                34|
|              Bahia|            22|                33|
|     Santa Catarina|            20|                33|
|     Rio de Janeiro|            15|                33|
|              Goiás|            18|                32|
|          São Paulo|            25|                29|
|            Roraima|            23|                28|
|              Amapá|            17|                27|
|               Acre|            22|                25|
|Rio Grande do Norte|            21|                25|
|               null|          null|            

#### Distribuição de cargos por nível

In [18]:
cargos_por_nivel = cargos_df.groupBy("Nível").count().withColumnRenamed("count", "Total").orderBy("Nível")
cargos_por_nivel.show()

+-----------+-----+
|      Nível|Total|
+-----------+-----+
|   Analista|    4|
|Coordenador|    4|
|    Diretor|    5|
|  Diretoria|    2|
| Estagiário|    5|
|    Gerente|    2|
|         RH|    3|
+-----------+-----+



#### Funcionários com maiores horas extras

In [19]:
funcionarios_hora_extra = pq_df.select("RG", "Nome completo", col("Horas extras").cast("int").alias("Horas extras")) \
                               .orderBy(desc("Horas extras"))

In [20]:
funcionarios_hora_extra.show()

+---------+--------------------+------------+
|       RG|       Nome completo|Horas extras|
+---------+--------------------+------------+
|369626839|Alfredo Machado A...|         200|
|307135080|Helena Carramanho...|         200|
|394177057|        Caio Stellet|         199|
|266341456| Brenno Ayello Leite|         199|
|395791459|        Izabel Costa|         198|
|684337006|Priscila de Oliveira|         198|
|311722917|    Wendela da Cunha|         198|
|909194922|       Leonardo Dias|         198|
|575413219| Mateus Novaes Silva|         197|
|698776875|Bárbara Spenchutt...|         196|
|762627169|André dos Santos ...|         196|
|952596883|Matheus Rodrigues...|         196|
|994059962| Renan Steiner Costa|         196|
|747804546|Daniel Pinto Anas...|         196|
|117694466|        Brenda Lopes|         195|
|762646893|    Rodrigo da Silva|         195|
|639216635|Thayná Freitas Me...|         195|
|396843893| Ana Lassaval Farias|         195|
|997859911|     Mauricio Jardim|  

#### Quantidade de Clientes e de Valor Contrato Anual por Nível (Dado o cargo responsável)

In [21]:
join_df = clientes_df.join(cargos_df, (clientes_df["Cargo Responsável"] == cargos_df["Cargo"]) & (clientes_df["Cargo Responsável"] == cargos_df["Cargo"]))

In [22]:
contrato_por_cargo = join_df.groupBy("Cargo", "Nível").agg({'Valor Contrato Anual': 'sum'})\
                            .withColumnRenamed("sum(Valor Contrato Anual)", "Valor Contrato Anual") \
                            .orderBy("Valor Contrato Anual", ascending=False)

In [23]:
contrato_por_cargo.show()

+-----+-----------+--------------------+
|Cargo|      Nível|Valor Contrato Anual|
+-----+-----------+--------------------+
|  OPV|    Diretor|           7479000.0|
| FIIV|  Diretoria|           5638500.0|
|FIIII|Coordenador|           5607000.0|
| COII|         RH|           5062500.0|
| FIII|   Analista|           4612500.0|
|COIII|         RH|           4504500.0|
| ADIV|         RH|           4428000.0|
|OPIII|Coordenador|           4428000.0|
|  ADV|    Diretor|           4414500.0|
| COIV|    Gerente|           3915000.0|
|  LOV|    Diretor|           3802500.0|
|LOIII|Coordenador|           3712500.0|
|ADIII|Coordenador|           3595500.0|
| OPII|   Analista|           3492000.0|
| LOII|   Analista|           3109500.0|
|  COV|    Diretor|           2880000.0|
| OPIV|    Gerente|           2763000.0|
| LOIV|  Diretoria|           2565000.0|
|  FIV|    Diretor|           2461500.0|
| ADII|   Analista|           1953000.0|
+-----+-----------+--------------------+



#### Clientes com maior Valor Contrato Anual

In [24]:
clientes_maior_contrato = clientes_df.withColumn("Valor Contrato Anual", col("Valor Contrato Anual").cast(FloatType()))\
                                        .orderBy("Valor Contrato Anual", ascending=False)

In [25]:
clientes_maior_contrato.show(10)

23/03/20 15:23:05 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Cliente, Valor Contrato Anual, Quantidade de Serviços, Cargo Responsável, CEP, Data Início Contrato, Nivel de Importancia, 
 Schema: Cliente, Valor Contrato Anual, Quantidade de Serviços, Cargo Responsável, CEP, Data Início Contrato, Nivel de Importancia, _c7
Expected: _c7 but found: 
CSV file: file:///Users/raquel.bustamante/Documents/Nestlé/Base_case/BaseClientes.csv
+----------------+--------------------+----------------------+-----------------+---------+--------------------+--------------------+----+
|         Cliente|Valor Contrato Anual|Quantidade de Serviços|Cargo Responsável|      CEP|Data Início Contrato|Nivel de Importancia| _c7|
+----------------+--------------------+----------------------+-----------------+---------+--------------------+--------------------+----+
|  Silva Quintana|            450000.0|                    33|              LOV|736840846|          24/03/2019|         